# BlazingContext API




.




## BlazingContext
[Docs](https://docs.blazingdb.com/docs/blazingcontext) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#BlazingContext)

In [ ]:
from blazingsql import BlazingContext

Create basic BlazingContext instance.

In [ ]:
bc = BlazingContext()

Configure some options when creating a BlazingContext instance.

In [ ]:
bc = BlazingContext(dask_client=None,
                    network_interface=None,
                    allocator='managed',
                    pool=False,
                    initial_pool_size=None,
                    enable_logging=False,
                    config_options={})

.


.


.

### Distributed SQL
[Docs](https://docs.blazingdb.com/docs/distributed) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Distributed-SQL)

Distribute BlazingSQL query execution across multiple GPUs with Dask. 

#### Single Node Multi-GPU
Distribute BlazingSQL query execution across all GPUs within the same system. 

Read more with our blog post "[Distributed SQL with Dask](https://blog.blazingdb.com/distributed-sql-with-dask-2979262acc8a?source=friends_link&sk=077319064cd7d9e18df8c0292eb5d33d)".

In [ ]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)

In [ ]:
from blazingsql import BlazingContext

bc = BlazingContext(dask_client=client, network_interface='lo')

.


.


.

## Storage Plugins

[Docs](https://docs.blazingdb.com/docs/connecting-data-sources) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Storage-Plugins)


In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

Register public AWS S3 bucket.

In [ ]:
bc.s3('bsql', bucket_name='blazingsql-colab')

.


.


.

## Create Tables
[Docs](https://docs.blazingdb.com/docs/creating-tables) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Create-Tables)


In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

Create table from local CSV file.

In [ ]:
bc.create_table('taxi', 'data/sample_taxi.csv', header=0)

Create table from local Parquet file.

In [ ]:
bc.create_table('taxi', 'data/sample_taxi.parquet')

Create table from Parquet file stored in public AWS S3 bucket.

In [ ]:
bc.s3('bsql', bucket_name='blazingsql-colab')

bc.create_table('taxi', 's3://bsql/yellow_taxi/taxi_data.parquet')

.


.


.

## Query Tables
[Docs](https://docs.blazingdb.com/docs/single-gpu) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Create-Tables)


In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

In [ ]:
bc.create_table('taxi', 'data/sample_taxi.parquet')

Pull all rows and all columns.

In [ ]:
bc.sql('SELECT * FROM taxi')

Determine the average number riders per trip by hour of the day.

In [ ]:
avg_riders_by_hour = '''
                     select
                         avg(cast(passenger_count as float)) as avg_passenger_count,
                         hour(dropoff_ts) as hour_of_the_day
                     from (
                         select
                             passenger_count, 
                             cast(tpep_dropoff_datetime || '.0' as timestamp) dropoff_ts
                         from
                             taxi
                             )
                     group by
                         hour(dropoff_ts)
                     order by
                         hour(dropoff_ts)
                         '''
bc.sql(avg_riders_by_hour)

.


.


.

# SQL Reference




.




## Data Types
[Docs](https://docs.blazingdb.com/docs/database-administration) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#SQL-Reference)

Data Types

These are the currently supported data types for BlazingSQL:

    Integer
        INT8
        INT16
        INT32
        INT64
    Floating Point
        FLOAT8
        FLOAT16
        FLOAT32
        FLOAT64
    Strings
        STR
    Date Time
        DATE32 (INT32_T DAYS since the UNIX Epoch)
        DATE64 (INT64_T MILLISECONDS since the UNIX Epoch)
        TIMESTAMP (INT64 encoded as exact TIMESTAMP since the UNIX Epoch)


.


.


.

## General SQL
[Docs](https://docs.blazingdb.com/docs/queries) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#General-SQL)

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

In [ ]:
bc.create_table('taxi', '../data/sample_taxi.parquet')

### SELECT

In [ ]:
bc.sql('SELECT * from taxi')

In [ ]:
bc.sql('SELECT passenger_count, trip_distance, index, fare_amount from taxi')

### WHERE
Query data and filter rows with a condition(s).

In [ ]:
# --Single Filter
bc.sql('select * from taxi WHERE passenger_count = 2')

In [ ]:
# --Multiple Filters (AND)
bc.sql('select * from taxi WHERE passenger_count > 2 AND trip_distance < 10')

In [ ]:
# --Multiple Filters (OR)
bc.sql('select * from taxi WHERE passenger_count >= 2 AND trip_distance <= 10')

In [ ]:
# --Multiple Filters (AND/OR)
bc.sql('select * from taxi WHERE (passenger_count <> 2 OR fare_amount > 100) AND trip_distance >= 10')

### GROUP BY
Groups result set of a query by one or more columns.

In [ ]:
bc.sql('select passenger_count from taxi GROUP BY passenger_count')

In [ ]:
query = '''
        select
            passenger_count,
            VendorID
        from
            taxi
        GROUP BY
            passenger_count, 
            VendorID
            '''
bc.sql(query)

#### SUM

In [ ]:
# --SUM without GROUP BY
bc.sql('select SUM(fare_amount) from taxi')

In [ ]:
# --SUM with GROUP BY
bc.sql('select SUM(fare_amount) from taxi GROUP BY passenger_count')

#### AVG

In [ ]:
# --AVG without GROUP BY
bc.sql('select AVG(fare_amount) from taxi')

In [ ]:
# --AVG with GROUP BY
bc.sql('select AVG(fare_amount) from taxi GROUP BY passenger_count')

#### MIN

In [ ]:
# --MIN without GROUP BY
bc.sql('select MIN(fare_amount) FROM taxi')

In [ ]:
# --MIN with GROUP BY
bc.sql('select MIN(fare_amount) from taxi GROUP BY passenger_count')

#### MAX

In [ ]:
# --MAX without GROUP BY
bc.sql('select MAX(fare_amount) FROM taxi')

In [ ]:
# --MAX with GROUP BY
bc.sql('select MAX(fare_amount) from taxi GROUP BY passenger_count')

#### COUNT

In [ ]:
# --COUNT without GROUP BY
bc.sql('select COUNT(fare_amount) from taxi')

In [ ]:
# --COUNT with GROUP BY
bc.sql('select COUNT(fare_amount) FROM taxi GROUP BY passenger_count')

#### DISTINCT

In [ ]:
# --COUNT without GROUP BY
bc.sql('select DISTINCT(fare_amount) from taxi')

In [ ]:
# --COUNT with GROUP BY
bc.sql('select DISTINCT(passenger_count) from taxi GROUP BY passenger_count')

### JOIN
Combine rows from two or more tables, based on a related column or columns between them.

#### INNER JOIN

In [ ]:
# --INNER JOIN SINGLE COLUMN
SELECT table_A.column_A, table_B.column_A, table_B.column_C 
FROM table_A 
INNER JOIN table_B 
ON (table_A.column_A = table_B.column_B)

In [ ]:
# --INNER JOIN MULTI-COLUMN
SELECT table_A.column_A, table_B.column_A, table_B.column_C 
FROM table_A 
INNER JOIN table_B 
ON (table_A.column_A = table_B.column_B AND table_A.column_C = table_B.column_C)

#### FULL OUTER JOIN

In [ ]:
# --FULL OUTER JOIN SINGLE COLUMN
SELECT table_A.column_A, table_B.column_A, table_B.column_C 
FROM table_A 
FULL OUTER JOIN table_B 
ON (table_A.column_A = table_B.column_B)

In [ ]:
# --FULL OUTER JOIN MULTI-COLUMN
SELECT table_A.column_A, table_B.column_A, table_B.column_C 
FROM table_A 
FULL OUTER JOIN table_B 
ON (table_A.column_A = table_B.column_B AND table_A.column_C = table_B.column_C)

#### LEFT OUTER

In [ ]:
# --LEFT JOIN SINGLE COLUMN
SELECT table_A.column_A, table_B.column_A, table_B.column_C 
FROM table_A 
LEFT JOIN table_B 
ON (table_A.column_A = table_B.column_B)

In [ ]:
# --LEFT JOIN MULTI-COLUMN
SELECT table_A.column_A, table_B.column_A, table_B.column_C 
FROM table_A 
LEFT JOIN table_B 
ON (table_A.column_A = table_B.column_B AND table_A.column_C = table_B.column_C)

### ORDER BY

In [ ]:
# --ORDER BY without Specificying (defaults to Ascending)
SELECT column_A, column_B FROM table_A ORDER BY column_A

In [ ]:
# -- ORDER BY Specifying Ascending
SELECT column_A, column_B FROM table_A ORDER BY column_A ASC

In [ ]:
# -- ORDER BY Specifying Descending
SELECT column_A, column_B FROM table_A ORDER BY column_A DESC

### NESTED QUERIES
A query nested inside another query that uses values from the outer query may appear elsewhere. 

In [ ]:
# -- NESTED QUERIES AS NEW TABLE 
SELECT column_A, column_B from
        (select avg(table_B.column_A) as column_A, max(table_B.column_B) as column_B, min(table_B.column_C) as column_C from table_B ) as InnerTable
        order by column_A, column_B

In [ ]:
# -- NESTED QUERIES AS NEW COLUMN
SELECT
   column_A,
   column_B,
   (SELECT AVG(column_D) 
      FROM table_A
      WHERE column_C = A.column_C) AS avg_column_D
   FROM table_A A

### CASE

In [ ]:
# --Single Filter
SELECT CASE WHEN column_A > 5 THEN 1 ELSE 0 FROM table_A

In [ ]:
# --Multiple Filters
SELECT 
    CASE WHEN column_A > 7 THEN 1 ELSE 0, 
    CASE WHEN column_B LIKE '%foo%' THEN 1 ELSE 0
FROM
    table_A

### CAST

In [ ]:
# --Single Filter
SELECT CAST(column_A AS type) FROM table_A

In [ ]:
# --Multiple Filters
SELECT 
    CAST(column_A as type) new_column_B, 
    CAST(column_B as type) new_column_A
FROM
    table_A

### LIKE

In [ ]:
# --Single Filter
SELECT * FROM table_A WHERE column_A LIKE '%foo%'

In [ ]:
# --Multiple Filters (AND)
SELECT 
    * 
FROM 
    table_A 
WHERE 
    column_A LIKE '%foo%' 
    AND column_B LIKE '%bar%'

In [ ]:
# --Multiple Filters (OR)
SELECT 
    * 
FROM 
    table_A 
WHERE 
    column_A LIKE '%foo%' 
    OR column_B LIKE '%bar%'

In [ ]:
# --Multiple Filters (AND/OR)
SELECT 
    * 
FROM 
    table_A 
WHERE 
    (column_A LIKE '%foo%' OR column_A LIKE '%bar%') 
    AND column_B LIKE '%bar%'

.


.


.

## Functions

### Unary Functions
[Docs](https://docs.blazingdb.com/docs/unary-ops) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Unary-Functions)

.


.


.

### Binary Functions
[Docs](https://docs.blazingdb.com/docs/binary-ops) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#Binary-Functions)

.


.


.

### DateTime Functions
[Docs](https://docs.blazingdb.com/docs/datetime-functions) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#DateTime-Functions)

.


.


.

### String Functions
[Docs](https://docs.blazingdb.com/docs/string-functions) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#String-Functions)

.


.


.

## Reserved Words
[Docs](https://docs.blazingdb.com/docs/reserved-words) | [BlazingSQL Notebooks](https://app.blazingsql.com/jupyter/user-redirect/lab/workspaces/auto-b/tree/Welcome_to_BlazingSQL_Notebooks/docs/blazingsql.ipynb#String-Functions)

BlazingSQL uses the Apache Calcite industry-standard SQL parser, validator and JDBC driver for SQL language. There is a common list of words that are reserved for the engine and, outside of their specified functionality, should not be used by users. 

You can find this list in:
- [Calcite Docs](https://calcite.apache.org/docs/reference.html#keywords)
- [Calcite Source Code](https://github.com/apache/calcite/blob/master/core/src/main/codegen/templates/Parser.jj#L6740)

.


.


.